<a href="https://colab.research.google.com/github/nanpolend/machine-learning/blob/master/%E8%83%8C%E6%99%AF%E5%81%B5%E6%B8%AC%E5%A4%9AFPS%E5%88%A4%E6%96%B7_Gemini%E4%BF%AE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
mermaid
graph LR
A[載入必要的函式庫] --> B{初始化背景減除器};
B --> C{開啟視訊來源};
C --> D{設定影像尺寸};
D --> E{主迴圈};
E --> F{讀取影像};
F --> G{調整影像尺寸};
G --> H{水平翻轉影像};
H --> I{套用背景減除器};
I --> J{二值化處理};
J --> K{膨脹處理};
K --> L{尋找輪廓};
L --> M{繪製輪廓和邊界框};
M --> N{轉換色彩空間};
N --> O{合併影像};
O --> P{顯示影像};
P --> Q{檢查是否按下ESC鍵};
Q -- 是 --> R[結束程式];
Q -- 否 --> E;

In [1]:
import cv2  # 載入 OpenCV 函式庫

# 初始化背景減除器
bs = cv2.bgsegm.createBackgroundSubtractorGMG()

# 開啟視訊來源 (® 應替換成實際的視訊檔案路徑或攝影機編號)
cap = cv2.VideoCapture(®)

# 計算影像長寬比並設定顯示尺寸
ratio = cap.get(cv2.CAP_PROP_FRAME_WIDTH) / cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
WIDTH = 400
HEIGHT = int(WIDTH / ratio)

# 主迴圈
while True:
    ret, frame = cap.read()  # 讀取影像
    if not ret:
        break # 若讀取失敗則跳出迴圈

    frame = cv2.resize(frame, (WIDTH, HEIGHT))  # 調整影像尺寸
    frame = cv2.flip(frame, 1)  # 水平翻轉影像

    # 套用背景減除器
    fgmask = bs.apply(frame)

    # 二值化處理
    _, mask = cv2.threshold(fgmask, 30, 255, cv2.THRESH_BINARY)

    # 膨脹處理 (形態學操作，用於消除雜訊)
    mask = cv2.dilate(mask, None, iterations=10)

    # 尋找輪廓
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # 繪製輪廓和邊界框
    for c in contours:
        if cv2.contourArea(c) < 200:  # 忽略太小的輪廓
            continue

        # 繪製輪廓
        cv2.drawContours(frame, [c], -1, (0, 255, 255), 2)

        # 繪製邊界框
        (x, y, w, h) = cv2.boundingRect(c)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # 轉換色彩空間 (將灰階遮罩轉換為彩色，以便合併顯示)
    mask = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)

    # 合併影像 (將原始影像和遮罩影像水平合併)
    frame = cv2.hconcat([frame, mask])

    # 顯示影像
    cv2.imshow('frame', frame)

    # 檢查是否按下ESC鍵
    if cv2.waitKey(1) == 27:
        break

# 關閉所有視窗
cv2.destroyAllWindows()

SyntaxError: invalid character '®' (U+00AE) (<ipython-input-1-154f996605e1>, line 7)